# Proyecto SCP
 * Jared Evander Fuentes Sánchez
 * Vicente Garza Reyna
 * Sebastián Terrazas Santillana

In [4]:
import cplex

In [ ]:
# Inicio del problema
def solveProblem(file_name):
    problem = new cplex.Cplex()
    # Leer Datos
    
    
    # Resolver el problema
    
    # Mostrar Solución
    

In [1]:
# Read File Loop